In [1]:
import numpy as np




In [2]:
in_data = np.load('data/dnn_base/in_data.npy')

layer1_output = np.load('data/dnn_base/layer1_output.npy')
layer2_output = np.load('data/dnn_base/layer2_output.npy')
layer3_output = np.load('data/dnn_base/layer3_output.npy')
layer4_output = np.load('data/dnn_base/layer4_output.npy')
layer5_output = np.load('data/dnn_base/layer5_output.npy')

activation1_output = np.load('data/dnn_base/activation1_output.npy')
activation2_output = np.load('data/dnn_base/activation2_output.npy')
activation3_output = np.load('data/dnn_base/activation3_output.npy')
activation4_output = np.load('data/dnn_base/activation4_output.npy')
activation5_output = np.load('data/dnn_base/activation5_output.npy')

In [3]:
print()
print("Model intermediate layer output")
print("==========================")
print()
print("in_data :", in_data.shape)
print("----------------------------------------------")
print("layer1_output :", layer1_output.shape)
print("----------------------------------------------")
print("activation1_output :", activation1_output.shape)
print("----------------------------------------------")
print("layer2_output :", layer2_output.shape)
print("----------------------------------------------")
print("activation2_output :", activation2_output.shape)
print("----------------------------------------------")
print("layer3_output :", layer3_output.shape)
print("----------------------------------------------")
print("activation3_output :", activation3_output.shape)
print("----------------------------------------------")
print("layer4_output :", layer4_output.shape)
print("----------------------------------------------")
print("activation4_output :", activation4_output.shape)
print("----------------------------------------------")
print("layer5_output :", layer5_output.shape)
print("----------------------------------------------")
print("activation5_output :", activation5_output.shape)


Model intermediate layer output

in_data : (60000, 784)
----------------------------------------------
layer1_output : (60000, 512)
----------------------------------------------
activation1_output : (60000, 512)
----------------------------------------------
layer2_output : (60000, 256)
----------------------------------------------
activation2_output : (60000, 256)
----------------------------------------------
layer3_output : (60000, 128)
----------------------------------------------
activation3_output : (60000, 128)
----------------------------------------------
layer4_output : (60000, 64)
----------------------------------------------
activation4_output : (60000, 64)
----------------------------------------------
layer5_output : (60000, 10)
----------------------------------------------
activation5_output : (60000, 10)


In [4]:
class RBFkernel():
    def __init__(self, sigma=0.5):
        self.sigma = sigma
        
    def __call__(self, x, y):
        numerator = -1 * np.linalg.norm(x - y, ord=2)**2
        denominator = 2 * (self.sigma**2)
        return np.exp(numerator / denominator)
    
    def get_params(self):
        return self.sigma
    
    def set_params(self, sigma):
        self.sigma = sigma
        
                
def gram_matrix(kernel, data, m):
    """
    Arguments:
    =========
    - kernel : kernel function 
    - data : data samples, shape=(m, dim(data_i))
    - m : number of samples
    """
    gram_matrix = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            gram_matrix[i][j] = kernel(data[i], data[j])
            
    print(gram_matrix) # debug code
    return gram_matrix


def hsic(k, l, m, X, Y):
    """
    Arguments:
    =========
    - k : kernel function for X
    - l : kernel function for Y
    - m : number of samples
    - X : data samples, shape=(m, dim(X_i))
    - Y : data samples, shape=(m, dim(Y_i))
    """
    H = np.diag(np.full(m, 1 - (1/m)))
    K = gram_matrix(k, X, m)
    L = gram_matrix(l, Y, m)
    HSIC = np.trace(np.dot(K, np.dot(H, np.dot(L, H)))) / ((m - 1)**2)
    return HSIC

In [5]:
# test code

kernel = RBFkernel(sigma=1)
kernel(np.array([1,2,3]), np.array([1,2,6]))

0.011108996538242306

## Measuring HSIC

In [ ]:
m = 30000

X = layer1_output[:m]
Y = layer2_output[:m]

kernel = RBFkernel(sigma=75)
hsic(kernel, kernel, m, X, Y)

In [ ]:
m = 10000

X = layer1_output[:m]
Y = layer2_output[:m]

kernel = RBFkernel(sigma=75)
hsic(kernel, kernel, m, X, Y)


[[1.         0.36167055 0.20541027 ... 0.32227845 0.3822559  0.40723468]
 [0.36167055 1.         0.2296683  ... 0.19952567 0.24814302 0.36743274]
 [0.20541027 0.2296683  1.         ... 0.19002902 0.41666704 0.31598154]
 ...
 [0.32227845 0.19952567 0.19002902 ... 1.         0.29799887 0.29246292]
 [0.3822559  0.24814302 0.41666704 ... 0.29799887 1.         0.47200955]
 [0.40723468 0.36743274 0.31598154 ... 0.29246292 0.47200955 1.        ]]


In [11]:
m = 3000

X = layer1_output[:m]
Y = layer2_output[:m]

kernel = RBFkernel(sigma=75)
hsic(kernel, kernel, m, X, Y)

[[1.         0.36167055 0.20541027 ... 0.43188578 0.38455003 0.56253039]
 [0.36167055 1.         0.2296683  ... 0.30816048 0.22918782 0.39305335]
 [0.20541027 0.2296683  1.         ... 0.41077349 0.42515682 0.32388919]
 ...
 [0.43188578 0.30816048 0.41077349 ... 1.         0.55576176 0.55646131]
 [0.38455003 0.22918782 0.42515682 ... 0.55576176 1.         0.46139165]
 [0.56253039 0.39305335 0.32388919 ... 0.55646131 0.46139165 1.        ]]
[[1.00000000e+00 8.92298008e-04 1.02908165e-03 ... 1.58852465e-02
  1.94432049e-02 1.76588538e-01]
 [8.92298008e-04 1.00000000e+00 3.69091668e-03 ... 7.30827527e-03
  7.08832915e-03 6.82750508e-04]
 [1.02908165e-03 3.69091668e-03 1.00000000e+00 ... 1.07502623e-01
  6.87234790e-02 5.00084891e-03]
 ...
 [1.58852465e-02 7.30827527e-03 1.07502623e-01 ... 1.00000000e+00
  1.45996268e-01 7.12428422e-02]
 [1.94432049e-02 7.08832915e-03 6.87234790e-02 ... 1.45996268e-01
  1.00000000e+00 2.25914285e-02]
 [1.76588538e-01 6.82750508e-04 5.00084891e-03 ... 7.124

0.04119317554515465

In [9]:
m = 1000

X = layer1_output[:m]
Y = layer2_output[:m]

kernel = RBFkernel(sigma=75)
hsic(kernel, kernel, m, X, Y)

[[1.         0.36167055 0.20541027 ... 0.26524436 0.38998084 0.29904935]
 [0.36167055 1.         0.2296683  ... 0.5193834  0.44600855 0.25464608]
 [0.20541027 0.2296683  1.         ... 0.23395897 0.30419092 0.29906306]
 ...
 [0.26524436 0.5193834  0.23395897 ... 1.         0.23777635 0.42172147]
 [0.38998084 0.44600855 0.30419092 ... 0.23777635 1.         0.33856887]
 [0.29904935 0.25464608 0.29906306 ... 0.42172147 0.33856887 1.        ]]
[[1.00000000e+00 8.92298008e-04 1.02908165e-03 ... 1.07123824e-03
  5.73648150e-02 1.13297109e-02]
 [8.92298008e-04 1.00000000e+00 3.69091668e-03 ... 4.35640761e-01
  3.00702072e-02 1.04532141e-02]
 [1.02908165e-03 3.69091668e-03 1.00000000e+00 ... 5.70902499e-03
  2.81826360e-02 3.50135361e-02]
 ...
 [1.07123824e-03 4.35640761e-01 5.70902499e-03 ... 1.00000000e+00
  2.50088313e-02 1.28505909e-02]
 [5.73648150e-02 3.00702072e-02 2.81826360e-02 ... 2.50088313e-02
  1.00000000e+00 5.39201786e-02]
 [1.13297109e-02 1.04532141e-02 3.50135361e-02 ... 1.285

0.04251413041022672

In [10]:
m = 1000

X = layer1_output[:m]
Y = layer2_output[:m]

kernel = RBFkernel(sigma=50)
hsic(kernel, kernel, m, X, Y)

[[1.         0.10143891 0.02840533 ... 0.05048982 0.12018419 0.06613349]
 [0.10143891 1.         0.03651551 ... 0.22900668 0.16256317 0.04606364]
 [0.02840533 0.03651551 1.         ... 0.03806837 0.06871935 0.06614031]
 ...
 [0.05048982 0.22900668 0.03806837 ... 1.         0.03948021 0.14332035]
 [0.12018419 0.16256317 0.06871935 ... 0.03948021 1.         0.08743913]
 [0.06613349 0.04606364 0.06614031 ... 0.14332035 0.08743913 1.        ]]
[[1.00000000e+00 1.37609155e-07 1.89675888e-07 ... 2.07607794e-07
  1.61046983e-03 4.18786197e-05]
 [1.37609155e-07 1.00000000e+00 3.35777848e-06 ... 1.54183942e-01
  3.76536384e-04 3.49391353e-05]
 [1.89675888e-07 3.35777848e-06 1.00000000e+00 ... 8.95909980e-06
  3.25430778e-04 5.30311372e-04]
 ...
 [2.07607794e-07 1.54183942e-01 8.95909980e-06 ... 1.00000000e+00
  2.48719675e-04 5.56002787e-05]
 [1.61046983e-03 3.76536384e-04 3.25430778e-04 ... 2.48719675e-04
  1.00000000e+00 1.40100755e-03]
 [4.18786197e-05 3.49391353e-05 5.30311372e-04 ... 5.560

0.008211154670520377

In [32]:
m = 2000

X = layer1_output[:m]
Y = layer2_output[:m]

kernel = RBFkernel(sigma=50)
hsic(kernel, kernel, m, X, Y)

[[1.         0.60595454 0.48592533 ... 0.55309902 0.63893733 0.1762112 ]
 [0.60595454 1.         0.52495055 ... 0.48570344 0.50872941 0.39230315]
 [0.48592533 0.52495055 1.         ... 0.27602119 0.5737177  0.18419574]
 ...
 [0.55309902 0.48570344 0.27602119 ... 1.         0.36398187 0.40916593]
 [0.63893733 0.50872941 0.5737177  ... 0.36398187 1.         0.16610418]
 [0.1762112  0.39230315 0.18419574 ... 0.40916593 0.16610418 1.        ]]
[[1.         0.15469112 0.15247753 ... 0.62371256 0.30955157 0.27937609]
 [0.15469112 1.         0.27570982 ... 0.17474942 0.39149234 0.76607383]
 [0.15247753 0.27570982 1.         ... 0.15648412 0.4741901  0.44373903]
 ...
 [0.62371256 0.17474942 0.15648412 ... 1.         0.26336744 0.31504925]
 [0.30955157 0.39149234 0.4741901  ... 0.26336744 1.         0.59731541]
 [0.27937609 0.76607383 0.44373903 ... 0.31504925 0.59731541 1.        ]]


0.28658106000122313

In [47]:
m = 100

X = layer1_output[:m]
Y = layer1_output[:m]

kernel = RBFkernel(sigma=500)
hsic(kernel, kernel, m, X, Y)

[[1.         0.99500302 0.99280898 ... 0.99639273 0.99714341 0.99468922]
 [0.99500302 1.         0.99357621 ... 0.99394681 0.99383381 0.99355107]
 [0.99280898 0.99357621 1.         ... 0.99344194 0.99510395 0.99515915]
 ...
 [0.99639273 0.99394681 0.99344194 ... 1.         0.99522313 0.99587812]
 [0.99714341 0.99383381 0.99510395 ... 0.99522313 1.         0.99563308]
 [0.99468922 0.99355107 0.99515915 ... 0.99587812 0.99563308 1.        ]]
[[1.         0.99500302 0.99280898 ... 0.99639273 0.99714341 0.99468922]
 [0.99500302 1.         0.99357621 ... 0.99394681 0.99383381 0.99355107]
 [0.99280898 0.99357621 1.         ... 0.99344194 0.99510395 0.99515915]
 ...
 [0.99639273 0.99394681 0.99344194 ... 1.         0.99522313 0.99587812]
 [0.99714341 0.99383381 0.99510395 ... 0.99522313 1.         0.99563308]
 [0.99468922 0.99355107 0.99515915 ... 0.99587812 0.99563308 1.        ]]


0.9882089671992826